In [74]:
import hdf5_getters
import os
import pyspark
import math
from pyspark.sql import SparkSession

In [75]:
# path="/scratch/work/public/MillionSongDataset/data"
def readSongData(filename):
    songlist = hdf5_getters.open_h5_file_read(filename)
    row = []
    #song related
    row.append(hdf5_getters.get_song_id(songlist).decode('UTF-8'))
    row.append(hdf5_getters.get_title(songlist).decode('UTF-8').lower())
    row.append(hdf5_getters.get_artist_id(songlist).decode('UTF-8'))
    hotness = hdf5_getters.get_song_hotttnesss(songlist)
    if hotness == hotness:
        row.append(hotness)
    else:
        row.append(-1)
    row.append(hdf5_getters.get_duration(songlist))
    row.append(hdf5_getters.get_end_of_fade_in(songlist))
    row.append(hdf5_getters.get_start_of_fade_out(songlist))
    row.append(len(hdf5_getters.get_segments_start(songlist)))
    row.append(len(hdf5_getters.get_beats_start(songlist)))
    row.append(len(hdf5_getters.get_sections_start(songlist)))
    row.append(len(hdf5_getters.get_tatums_start(songlist)))
    return [str(r) for r in row]

def readArtistsData(filename):
    songlist = hdf5_getters.open_h5_file_read(filename)
    row = []
    #artist data
    row.append(hdf5_getters.get_artist_id(songlist).decode('UTF-8'))
    row.append(hdf5_getters.get_artist_name(songlist).decode('UTF-8'))
    row.append(hdf5_getters.get_artist_familiarity(songlist))
    row.append(hdf5_getters.get_artist_hotttnesss(songlist))
    lat = hdf5_getters.get_artist_latitude(songlist)
    long = hdf5_getters.get_artist_longitude(songlist)
    if lat == lat:
        row.append(lat)
    else:
        row.append(1000)
    if long == long:
        row.append(long)
    else:
        row.append(1000)
    sim_artist = hdf5_getters.get_similar_artists(songlist)[:20]
    temp = []
    for ar in sim_artist:
        temp.append(ar.decode('UTF-8'))
    artist_terms = hdf5_getters.get_artist_terms(songlist)
    row.append(','.join(temp))
    val = min(len(artist_terms), 10)
    terms = ','.join([term.decode('UTF-8') for term in artist_terms[:val]])
    row.append(terms)
    artist_weights = hdf5_getters.get_artist_terms_weight(songlist)
    terms = ','.join([str(term) for term in artist_weights[:val]])
    row.append(terms)
    return [str(r) for r in row]

In [3]:
arr = ["similar_artists_"+str(i+1) for i in range(20)]

In [4]:
artist_columns = ["id","name","familiarity","hotttnesss","latitude","longitude"]+arr+["artist_terms","artist_weights"]

In [14]:
spark = SparkSession.builder.appName('MillionSongProcess').getOrCreate()
files = []

In [15]:
for x,y,z in os.walk(path):
    for f in z:
        files.append(x+"/"+f)

In [16]:
len(files)

1000000

In [17]:
# artist_rdd = spark.sparkContext.parallelize(files).map(lambda x: readArtistsData(x))

In [ ]:
# artistDF = spark.createDataFrame(artist_rdd, artist_columns)

In [ ]:
# artistDF.show()

In [24]:
# artistDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- familiarity: string (nullable = true)
 |-- hotttnesss: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- similar_artists_1: string (nullable = true)
 |-- similar_artists_2: string (nullable = true)
 |-- similar_artists_3: string (nullable = true)
 |-- similar_artists_4: string (nullable = true)
 |-- similar_artists_5: string (nullable = true)
 |-- similar_artists_6: string (nullable = true)
 |-- similar_artists_7: string (nullable = true)
 |-- similar_artists_8: string (nullable = true)
 |-- similar_artists_9: string (nullable = true)
 |-- similar_artists_10: string (nullable = true)
 |-- similar_artists_11: string (nullable = true)
 |-- similar_artists_12: string (nullable = true)
 |-- similar_artists_13: string (nullable = true)
 |-- similar_artists_14: string (nullable = true)
 |-- similar_artists_15: string (nullable = true)
 |-- similar_artis

In [25]:
# artistDF.write.parquet("artistDFA.parquet")

22/04/30 01:19:46 ERROR Utils: Aborting task                      (0 + 96) / 96]
java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 28 fields are required while 8 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:186)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:211)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:180)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:750)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.Who

Py4JJavaError: An error occurred while calling o141.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at or

22/04/30 01:19:47 ERROR Utils: Aborting task
org.apache.spark.TaskKilledException
	at org.apache.spark.TaskContextImpl.killTaskIfInterrupted(TaskContextImpl.scala:216)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:36)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.writeWithIterator(FileFormatDataWriter.scala:91)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeTask$1(FileFormatWriter.scala

22/04/30 01:19:48 WARN TaskSetManager: Lost task 35.0 in stage 8.0 (TID 233) (hlog-2.hpc.nyu.edu executor driver): TaskKilled (Stage cancelled)
22/04/30 01:19:48 WARN TaskSetManager: Lost task 15.0 in stage 8.0 (TID 213) (hlog-2.hpc.nyu.edu executor driver): TaskKilled (Stage cancelled)
22/04/30 01:19:48 ERROR FileFormatWriter: Job job_202204300119436194062239641715996_0008 aborted.
22/04/30 01:19:48 ERROR Utils: Aborting task
org.apache.spark.TaskKilledException
	at org.apache.spark.TaskContextImpl.killTaskIfInterrupted(TaskContextImpl.scala:216)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:36)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.Buff

22/04/30 01:19:48 ERROR Utils: Aborting task
org.apache.spark.TaskKilledException
	at org.apache.spark.TaskContextImpl.killTaskIfInterrupted(TaskContextImpl.scala:216)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:36)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.writeWithIterator(FileFormatDataWriter.scala:91)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeTask$1(FileFormatWriter.scala

In [39]:
def outputSongs(letter):
    path="/scratch/work/public/MillionSongDataset/data/"+letter
    print(path)
    files = []
    for x,y,z in os.walk(path):
        for f in z:
            files.append(x+"/"+f)
    song_rdd = spark.sparkContext.parallelize(files).map(lambda x: readSongData(x))
    song_columns = ["id","title","artist_id","hotttnesss","duration","fade_in","fade_out","nsegments","nbeats","nsections","ntatums"]
    songsDF = spark.createDataFrame(song_rdd, song_columns)
    songsDF.write.parquet("songDF"+letter+".parquet")
    

In [76]:
def outputArtists(letter):
    path="/scratch/work/public/MillionSongDataset/data/"+letter
    print(path)
    files = []
    for x,y,z in os.walk(path):
        for f in z:
            files.append(x+"/"+f)
    artist_rdd = spark.sparkContext.parallelize(files).map(lambda x: readArtistsData(x))
    arr = ['similar_artists']
    artist_columns = ["id","name","familiarity","hotttnesss","latitude","longitude"]+arr+["artist_terms","artist_weights"]
    songsDF = spark.createDataFrame(artist_rdd, artist_columns)
    songsDF.write.parquet("artistDF"+letter+".parquet")

In [77]:
import string
liss = list(string.ascii_uppercase)
liss[18:]

['S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [78]:
for l in liss[18:]:
    outputArtists(l)

/scratch/work/public/MillionSongDataset/data/S


22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 90.04% for 8 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 80.03% for 9 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 72.03% for 10 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 65.48% for 11 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 60.02% for 12 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 55.41% for 13 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760

22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.92% for 91 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.83% for 92 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.75% for 93 writers
22/04/30 03:52:59 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.66% for 94 writers
22/04/30 03:53:00 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.58% for 95 writers
22/04/30 03:53:00 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.50% for 96 writers
22/04/30 03:53:08 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845

22/04/30 03:53:09 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 16.37% for 44 writers
22/04/30 03:53:09 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 16.75% for 43 writers
22/04/30 03:53:09 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 17.15% for 42 writers
22/04/30 03:53:09 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 17.57% for 41 writers
22/04/30 03:53:09 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 18.01% for 40 writers
22/04/30 03:53:09 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 18.47% for 39 writers
22/04/30 03:53:09 WARN MemoryManager: Total allocation exceeds 95.00% (966,7

/scratch/work/public/MillionSongDataset/data/T


...

22/04/30 03:54:41 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 51.45% for 14 writers
22/04/30 03:54:41 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 55.41% for 13 writers
22/04/30 03:54:41 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 60.02% for 12 writers
22/04/30 03:54:41 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 65.48% for 11 writers
22/04/30 03:54:41 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 72.03% for 10 writers
22/04/30 03:54:41 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 80.03% for 9 writers
22/04/30 03:54:41 WARN MemoryManager: Total allocation exceeds 95.00% (966,76

/scratch/work/public/MillionSongDataset/data/Z


22/04/30 03:54:46 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 90.04% for 8 writers
22/04/30 03:54:47 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 80.03% for 9 writers
22/04/30 03:54:47 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 72.03% for 10 writers
22/04/30 03:54:47 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 65.48% for 11 writers
22/04/30 03:54:47 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 60.02% for 12 writers
22/04/30 03:54:47 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 55.41% for 13 writers
22/04/30 03:54:47 WARN MemoryManager: Total allocation exceeds 95.00% (966,760

22/04/30 03:54:47 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.58% for 95 writers
22/04/30 03:54:47 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.50% for 96 writers
22/04/30 03:54:55 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.58% for 95 writers
22/04/30 03:54:56 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.66% for 94 writers
22/04/30 03:54:56 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.75% for 93 writers
22/04/30 03:54:56 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 7.83% for 92 writers
22/04/30 03:54:56 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845

22/04/30 03:54:57 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 25.72% for 28 writers
22/04/30 03:54:57 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 26.68% for 27 writers
22/04/30 03:54:57 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 27.70% for 26 writers
22/04/30 03:54:57 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 28.81% for 25 writers
22/04/30 03:54:57 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 30.01% for 24 writers
22/04/30 03:54:57 WARN MemoryManager: Total allocation exceeds 95.00% (966,760,845 bytes) of heap memory
Scaling row group sizes to 31.32% for 23 writers
22/04/30 03:54:57 WARN MemoryManager: Total allocation exceeds 95.00% (966,7